# Python 101 @ SzISz VII.

---

## Today: Web Scraping II.

### Act I: scrape the hungarian tenders

In [3]:
# import the necessary libraries
import requests
from bs4 import BeautifulSoup

Let's get some nasty data!

In [2]:
# store some general data
BASE_URL = 'http://kozbeszerzes.ceu.hu'
kozgep_suburl = '/entity/t/10950676.xml'

In [3]:
# download raw data
kozgep_response = requests.get(BASE_URL + kozgep_suburl)
print kozgep_response.status_code

200


In [4]:
# print what did we get
soup = BeautifulSoup(kozgep_response.content)
print soup.prettify()

<html>
 <body>
  <entity city="Budapest" id="t_10950676" name="KÖZGÉP Zrt" url="/entity/t/10950676.xml">
   <all_known_names>
    <name>
     KÖZGÉP Építő- és Fémszerkezetgyártó Zrt,
    </name>
    <name>
     Közgép Építő - és Fémszerkezetgyártó Zrt.
    </name>
    <name>
     Közgép Zrt.
    </name>
    <name>
     KÖZGÉP-UNIÓ Gép- és Fémszerkezetgyártó Rt.
    </name>
    <name>
     KÖZGÉP Építő és Fémszerkezetgyártó Zrt.
    </name>
    <name>
     Magyarország;KÖZGÉP Zrt.
    </name>
    <name>
     Közép Építő- és Fémszerkezetgyártó Zrt.
    </name>
    <name>
     KÖZGÉP Gép- és Fémszerkezetgyártó Rt.
    </name>
    <name>
     Tagok: KÖZGÉP Zrt.
    </name>
    <name>
     KÖZÉP-UNIÓ Rt.
    </name>
    <name>
     Magyarország KÖZGÉP Zrt.
    </name>
    <name>
     Konzorciumvezető: Közgép Építő- és Fémszerkezetgyártó Zrt.
    </name>
    <name>
     Közgép Építő, és Fémszerkezetgyártó Zrt.
    </name>
    <name>
     Közgép Építő-, és Fémszerkezetgyártó Zrt.
    </name>


In [5]:
# iterate through the tenders
# beautifulsoup can parse xmls too
for tender in soup.find('all_tenders_won').findAll('tender'):
    print BASE_URL + tender['url']

http://kozbeszerzes.ceu.hu/tender/2013/00/157.xml
http://kozbeszerzes.ceu.hu/tender/2012/00/11695.xml
http://kozbeszerzes.ceu.hu/tender/2011/00/8803.xml
http://kozbeszerzes.ceu.hu/tender/2007/00/15485.xml
http://kozbeszerzes.ceu.hu/tender/2002/28/5592.xml
http://kozbeszerzes.ceu.hu/tender/2013/00/1360.xml
http://kozbeszerzes.ceu.hu/tender/2013/00/23476.xml
http://kozbeszerzes.ceu.hu/tender/2011/00/7654.xml
http://kozbeszerzes.ceu.hu/tender/2013/00/2812.xml
http://kozbeszerzes.ceu.hu/tender/2012/00/9199.xml
http://kozbeszerzes.ceu.hu/tender/2010/00/7735.xml
http://kozbeszerzes.ceu.hu/tender/2013/00/2422.xml
http://kozbeszerzes.ceu.hu/tender/2013/00/150.xml
http://kozbeszerzes.ceu.hu/tender/2002/24/4337.xml
http://kozbeszerzes.ceu.hu/tender/2011/00/31190.xml
http://kozbeszerzes.ceu.hu/tender/2009/00/26598.xml
http://kozbeszerzes.ceu.hu/tender/2012/00/5937.xml
http://kozbeszerzes.ceu.hu/tender/2013/00/6185.xml
http://kozbeszerzes.ceu.hu/tender/2012/00/10718.xml
http://kozbeszerzes.ceu.hu/

In [6]:
# parse the first tender
tender_response = requests.get(
    BASE_URL + soup.find('all_tenders_won').find('tender')['url']
)
if not tender_response.status_code == 200:
    print 'Tender download failed!'
else:
    tender_soup = BeautifulSoup(tender_response.content)
    print tender_soup.prettify()

<html>
 <body>
  <tender estimated_value="400000000" id="2013_00_157" requestor_city="Budapest" requestor_name="MÁV Rt" source_url="http://www.kozbeszerzes.hu/adatbazis/mutat/hirdetmeny/portal_157_2013" subject="A MÁV Zrt. vasúti hidak és acélszerkezetek felújítása és állagmegóvása - kivitelezés tárgyú keretmegállapodás alapján az alábbi munkák vállalkozásba adása : 5B. RÉSZ (MÁV VII. csomag / B rész)" url="/tender/2013/00/157.xml" year="2013">
   <proceduretype>
    Ajánlati/részvételi felhívás közzététele nélküli/hirdetmény nélküli tárgyalásos
   </proceduretype>
   <estimatedvalue>
    400000000
   </estimatedvalue>
   <press>
    2013/5
   </press>
   <requestor city="Budapest" id="t_10856417" name="MÁV Rt" url="/entity/t/10856417.xml">
   </requestor>
   <parts>
    <part id="2013_00_157_1">
     <allbidders>
     </allbidders>
     <currency>
      HUF
     </currency>
     <decisiondate>
      2012/12/14
     </decisiondate>
     <estimatedvalue>
      400000000
     </estimated

In [7]:
# create a function, and get the needed information out of the xml
def get_tenders(base_url, sub_url):
    response = requests.get(base_url + sub_url)
    if not response.status_code == 200:
        print 'Download failed!'
    else:
        won_tenders = [['Year', 'Value', 'Desc']] # init with headers
        soup = BeautifulSoup(response.content)
        for tender in soup.find('all_tenders_won').findAll('tender'):
            tender_response = requests.get(base_url + tender['url'])
            if not tender_response.status_code == 200:
                print 'Tender download failed!'
            else:
                tender_soup = BeautifulSoup(tender_response.content)
                won_tenders.append([
                    tender_soup.find('tender')['year'],
                    tender_soup.find('tender')['estimated_value'],
                    '"' + tender_soup.find('tender')['subject'] + '"' # we use " to make sure that the data is wrapped
                ])
        return won_tenders

In [8]:
# write a save function
# since we have hungarian text, we need to encode our characters in UTF-8
# and unfortunately csv module does not support that
import codecs
def save_results(filename, tenders):
    with codecs.open(filename, 'w', 'utf-8') as output:
        for tender in tenders:
            output.write(u';'.join(tender) + u'\n')

In [9]:
# write a main function
def main():
    save_results(BASE_URI + 'kozgep.csv', get_tenders(BASE_URL, kozgep_suburl))

In [10]:
# execute
main()

---

### Intermission: Creating a standalone script

In [11]:
# Intermission
from IPython.display import YouTubeVideo
YouTubeVideo("O0wOD9TWynM", autoplay=1)

Create a new text file with .py extension! You can specify the filename.
Start it with:  
    `# encoding: utf-8`  
then copy-paste:
    - the imports, 
    - the global variables 
    - the three functions
and insert the following two lines into the end of the file:  
`if __name__ == '__main__':  
     main()`  
Save it, and now you can execute this script by invoking:  
    `python your_specified_filename.py`

In [12]:
# You can even import your newly created script:
import myscript # use your filename

In [13]:
# get it's contents
dir(myscript)

['BASE_URL',
 'BeautifulSoup',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 'codecs',
 'get_tenders',
 'kozgep_suburl',
 'main',
 'requests',
 'save_results']

In [14]:
# print its variables
print myscript.BASE_URL

http://kozbeszerzes.ceu.hu


In [15]:
# use its functions
tenders = myscript.get_tenders(myscript.BASE_URL, myscript.kozgep_suburl)

In [16]:
myscript.save_results(BASE_URI + 'kozgep1.csv', tenders)

---

### Act II: Disguise yourself!

Let's pretend to be a browser instead of a script

In [1]:
USER_AGENTS = [
    # Chrome
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
    # Firefox
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:25.0) Gecko/20100101 Firefox/25.0',
    # Opera
    'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
    # Safari
    'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25',
    # Internet Explorer, probably a good idea to leave this one out...
    'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
]

In [2]:
# write a wrapper function to handle the user-agent string
import random
def get_header(agents):
    return {'User-agent': random.choice(agents)}

Get the main articles from index.hu

In [4]:
url = 'http://index.hu'
index_response = requests.get(url, headers=get_header(USER_AGENTS))

In [5]:
# study the skeleton of the site
soup = BeautifulSoup(index_response.content)
print soup.prettify()

<!DOCTYPE html>
<html lang="hu" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
 <head>
  <title>
   Index
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="hu-hu" http-equiv="Content-Language"/>
  <!-- [if IE]>
    <meta http-equiv="imagetoolbar" content="no" />
    <meta name="MSSmartTagsPreventParsing" content="true" />
    <meta http-equiv="X-UA-Compatible" value="IE=Edge">
    <![endif]-->
  <meta content="ALL" name="ROBOTS"/>
  <meta content="http://index.hu/copyright/" name="copyright"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="330827937019153" property="fb:app_id"/>
  <meta content="593022362" property="fb:admins"/>
  <meta content="http://index.hu/assets/images/facebook_logo.png" property="og:image"/>
  <meta content="https://www.facebook.com/Indexhu" property="article:publisher"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="@indexhu" nam

In [9]:
# get the "front page"
index_main = soup.find('section', {'class': 'blokk hajtas-felett dupla-vezeto-blokk cimlap-blokk-index saved'})

In [11]:
# print the basics about the main articles
for article in index_main.findAll('article'):
    print ''
    # article image if exists
    if article.find('img'):
        print '[', article.find('img').get('src'), ']'
    # title
    print article.find('h1', {'class': 'cikkcim'}).getText()
    # link
    print '<', article.find('h1', {'class': 'cikkcim'}).find('a').get('href'), '>'
    # promo text if exists
    if article.find('p', {'class': 'ajanlo'}):
        print article.find('p', {'class': 'ajanlo'}).getText()
    print '-' * 79


[ http://kep.cdn.index.hu/1/0/831/8310/83104/8310497_181713_790fdd7e05f3903b6e7f809d33d94140_wm.jpg ]

Ki is itt a kenyérkereső?

< http://dex.hu/x.php?id=inxcl&url=http%3A%2F%2Fuvegplafon.blog.hu%2F2015%2F05%2F03%2Fki_is_itt_a_kenyerkereso >

Az Üvegplafon anyák napi műsora arról, hogy mi köze a vasalásnak a nemek egyenjogúságához, és hogy miként boldogul egy átlagosnak mondható család külső segítség nélkül.

-------------------------------------------------------------------------------


Háromszáz foglyot végzett ki az Iszlám Állam

< http://dex.hu/x.php?id=inxcl&url=http%3A%2F%2Findex.hu%2Fkulfold%2F2015%2F05%2F03%2Fharomszaz_foglyot_vegzett_ki_az_iszlam_allam%2F >

Még több ezer foglyuk lehet. Az ENSZ szerint népirtás folyik a jazidi kisebbség ellen.

-------------------------------------------------------------------------------


Tízmilliárdokat bukott, azt mondta, nem nagy ügy

< http://dex.hu/x.php?id=inxcl&url=http%3A%2F%2Findex.hu%2Fgazdasag%2F2015%2F05%2F03%2Fhig%2F >

Eze

Let's get the article texts, and the list of images for each "main" article!

In [12]:
articles = []
for article in index_main.findAll('article'):
    article_response = requests.get(
        article.find('h1', {'class': 'cikkcim'}).find('a').get('href'),
        headers=get_header(USER_AGENTS)
    )
    soup = BeautifulSoup(article_response.content)
    article_container = soup.find('div', {'class':'cikk-torzs-container'})
    if article_container:
        title = article.find('h1', {'class': 'cikkcim'}).getText()
        text = u'\n'.join([p.getText() for p in article_container.findAll('p')])
        images = [url.get('src') for url in article_container.findAll('img')]
        articles.append([title, text, images])

In [13]:
for article in articles:
    print ''
    print 'Title:', article[0]
    print 'Text:', article[1]
    print 'Images:'
    for img in article[2]:
        print img
    print '-' * 79


Title: 
Háromszáz foglyot végzett ki az Iszlám Állam

Text: A jazidi kisebbséghez tartozó háromszáz foglyot végeztek ki az Iszlám Állam terrorszervezet fegyveresei az iraki Moszul közelében - közölte az MTI jazidi és iraki forrásokra hivatkozva. A BBC idézte vasárnapra virradóra a Jazidi Haladó Párt közleményét, amely szerint a foglyokat pénteken ölték meg a Moszul melletti Tal Afar körzetben. A párt elítélte a tömegmészárlást, iszonyatos bűnténynek nevezte, és felszólította az iraki erőket, hogy szabadítsák ki azokat a jazidikat, akik még mindig az Iszlám Állam fogságában vannak. Oszama al-Nudzsaifi iraki alelnök borzalmasnak és barbárnak nevezte a mészárlást.
Az Iszlám Állam fegyveresei tavaly jazidik ezreit ejtették foglyul, és állítólag a fellegváruknak számító Moszulban tartják őket fogva. Április elején, nyolc hónapnyi fogva tartás után közülük több mint kétszázat engedtek szabadon. Az Iszlám Állam fegyveresei jazidik százait ölték meg az általuk elfoglalt területeken. Az ENSZ E

### Final Act: Your turn!

Write a script called `youtube.py`, in which you create an object called RelatedTube.
It has an attribute: `base_url` (youtube's base url)
It has three functions: `init`, `get`, and `set`

Init:
    - Arguments: (`self` and) `youtube_video_id`
    - Output: -
    - Workflow: set the `self.video` to `youtube_video_id`
Get:
    - Arguments: `self`
    - Output: the links to the related videos
    - Workflow: 
        * get the `self.video` page
        * parse it for the related links
        * return them in a list
Set:
    - Arguments: (`self` and) `youtube_video_id`
    - Output: -
    - Workflow: set the `self.video` to `youtube_video_id`
Don't forget to hide your a**!!!

In [15]:
# test the script
import youtube

In [28]:
related = youtube.RelatedTube('zkxqRthhwIs')

In [32]:
for video in related.get():
    print video

http://youtube.com/watch?v=WNbviyEOxsI
http://youtube.com/watch?v=2TCXpgL9VY8
http://youtube.com/watch?v=_w78oJ1BFDA
http://youtube.com/watch?v=IVjOBvnqfTQ
http://youtube.com/watch?v=41aGCrXM20E
http://youtube.com/watch?v=KYniUCGPGLs
http://youtube.com/watch?v=HP-MbfHFUqs
http://youtube.com/watch?v=REtFmT4Y-ns
http://youtube.com/watch?v=PS_T3Jg3uaY
http://youtube.com/watch?v=gZSWCU44wkw
http://youtube.com/watch?v=x_nZaS372Ew
http://youtube.com/watch?v=3p_v8IRaL20
